In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter

%matplotlib inline

/home/samuel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#race = "white"
#race = "hispanic"
race = "mixed"

In [3]:
if race == "white":
    data = pd.read_csv("../Data/white/white_2year.csv")
if race == "hispanic":
    data = pd.read_csv("../Data/hispanic/hispanic_2year.csv")
if race == "mixed":
    data = pd.read_csv("../Data/mixed/mixed_2year.csv")
data_matrix = data.as_matrix()

In [4]:
data.head()

,pubcsnum,mar_stat,sex,primsite,histo3v,beho3v,grade,dx_conf,csexten,cslymphn,...,eod10_pn,eod10_ne,cstumsiz,maligcount,benbordcount,year_dx,nhiade,codpub,srv_time_mon,srv_time_mon_flag
0,7000085,5,1,C199,8210,3,2,1,160,0,...,98,0,999,4,0,2007,0,21040,47,1
1,7000085,9,1,C180,8140,3,2,1,600,300,...,97,98,40,4,0,2007,0,21040,47,1
2,7000321,5,2,C187,8210,3,1,1,200,0,...,0,2,8,2,0,2012,0,0,39,1
3,7000411,5,2,C187,8480,3,2,1,450,200,...,1,11,75,2,0,2006,0,21040,38,1
4,7000472,5,2,C199,8020,3,4,1,999,0,...,98,0,999,3,0,2009,0,21050,0,1


In [20]:
ethnicity_labels = np.clip(data_matrix[:,-4],0,1)
if race == "mixed":
    np.save('../Data/mixed/ethnicity_labels.npy',ethnicity_labels)

In [5]:
data_matrix.shape

(313122, 29)

In [6]:
data_matrix[0]

array([7000085, 5, 1, 'C199', 8210, 3, 2, 1, 160, 0, 0, 3.0, 0.0, 0.0, 26,
       0, 1, 1502, 77, 98, 0, 999, 4, 0, 2007, 0, 21040, 47, 1],
      dtype=object)

In [7]:
var_names = []
with open('var_names.txt', 'r') as f_vars:
    for name in f_vars:
        var_names.append(name.strip())

In [8]:
var_names

['mar_stat',
 'sex',
 'primsite',
 'histo3v',
 'beho3v',
 'grade',
 'dx_conf',
 'csexten',
 'cslymphn',
 'csmetsdx',
 'cstseval',
 'csrgeval',
 'csmteval',
 'surgprif',
 'no_surg',
 'summ2k',
 'reg',
 'age_dx',
 'eod10_pn',
 'cstumsiz',
 'maligcount']

# Preprocessing

In [9]:
# Y2 will be indicator of survival; 21040 and 21050 are SEER codes for colon and rectum cancer
Y2 = np.logical_not(np.logical_and(data_matrix[:,-2].astype(int)<24, np.isin(data_matrix[:,-3], (21040, 21050))))
X = data_matrix[:,1:23]
X_cont = X[:,-5:]
X_cat = X[:,:-5]
X_cat.shape

(313122, 17)

In [10]:
X_cont[0]

array([77, 98, 0, 999, 4], dtype=object)

In [11]:
# Fill missing values in three eval columns with 9: unknown
# remember: index shifted over one since we deleted primsite <-- no longer the case
X_cat[:,12] = np.array([9 if np.isnan(x) else x for x in X_cat[:,12]])
X_cat[:,10] = np.array([9 if np.isnan(x) else x for x in X_cat[:,10]])
X_cat[:,11] = np.array([9 if np.isnan(x) else x for x in X_cat[:,11]])

In [12]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.externals import joblib

features = {}

X_cat_bin = np.zeros((X_cat.shape[0], 1))
start_idx = 0

for i in range(X_cat.shape[1]):
    lb = LabelBinarizer()
    X_cat_bin = np.hstack((X_cat_bin, lb.fit_transform(X_cat[:,i].astype(str))))
    features[(i, var_names[i],start_idx)] = lb.classes_
    start_idx += len(lb.classes_)
    if len(lb.classes_)==2:
        start_idx-=1
    #if race != 'mixed': #used all-at-once OneHotEncoder instead of separate LabelBinarizers
        #if len(lb.classes_) == 2:
            #start_idx -= 1
X_cat_bin = np.delete(X_cat_bin, [0], axis=1)
X_cat = X_cat_bin

In [13]:
import pickle
with open('ethnicity_codes/' + race + '_codes.pkl', 'wb') as f:
    pickle.dump(features, f, pickle.HIGHEST_PROTOCOL)

In [14]:
features

{(0, 'mar_stat', 0): array(['1', '2', '3', '4', '5', '6', '9'], dtype='<U1'),
 (1, 'sex', 7): array(['1', '2'], dtype='<U1'),
 (2,
  'primsite',
  8): array(['C180', 'C181', 'C182', 'C183', 'C184', 'C185', 'C186', 'C187',
        'C188', 'C189', 'C199', 'C209', 'C260'], dtype='<U4'),
 (3,
  'histo3v',
  21): array(['8000', '8001', '8002', '8003', '8004', '8010', '8011', '8012',
        '8013', '8015', '8020', '8021', '8022', '8030', '8031', '8032',
        '8033', '8041', '8045', '8046', '8050', '8051', '8052', '8070',
        '8071', '8072', '8073', '8076', '8077', '8081', '8082', '8083',
        '8094', '8123', '8124', '8130', '8140', '8141', '8142', '8143',
        '8144', '8145', '8147', '8160', '8201', '8210', '8211', '8213',
        '8220', '8221', '8230', '8240', '8241', '8243', '8244', '8245',
        '8246', '8249', '8255', '8260', '8261', '8262', '8263', '8310',
        '8320', '8323', '8341', '8380', '8430', '8440', '8441', '8460',
        '8461', '8470', '8471', '8472', '84

In [13]:
X_cat_bin.shape

(313122, 368)

The continuous variables in order: age_dx, eod10_pn, eod10_ne, cstumsiz, maligcount. Age and maligcount do not require imputation, and eod10_ne (nodes examined) will not be used as a variable.


Here we will impute values for eod10_pn and cstumsiz. We will implement multivariate imputation by chained equations (MICE) on these two continuous variables. Their indices are and values which need to be imputed are:

1: eod10_pn (95, 97, 98, 99)<br>
3: cstumsiz (998, 999)<br>

Additionally, we will replace the values (991, 992, 993, 994, 995) in cstumsiz with (1, 2, 3, 4, 5) respectively.


### Note: The above should be re-verified for each different dataset. This is for colorectal cancer in Hispanic patients after 2004.

In [19]:
# implements MICE imputation with linear regression model on eod10_pn and cstumsiz
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression

NUM_CYCLES=10

missing_values_pn = (95, 97, 98, 99)
missing_values_tumsiz = (998, 999)

pn = np.array(X_cont[:,1]).reshape(-1,1)
tumsiz = np.array(X_cont[:,3]).reshape(-1,1)

# non-MICE imputation
for i, size in enumerate(tumsiz):
    if size not in (990, 991, 992, 993, 994, 995):
        continue
    elif size == 990:
        tumsiz[i] = 0
    elif size == 991:
        tumsiz[i] = 1
    elif size == 992:
        tumsiz[i] = 2
    elif size == 993:
        tumsiz[i] = 3
    elif size == 994:
        tumsiz[i] = 4
    elif size == 995:
        tumsiz[i] = 5


# set missing values to -1
for i, value in enumerate(pn):
    if value in missing_values_pn:
        pn[i] = -1
for i, value in enumerate(tumsiz):
    if value in missing_values_tumsiz:
        tumsiz[i] = -1

# indices for valid and imputation-needed positions in pn and tumsiz
idx_pn_missing = np.where(pn == -1)[0]
idx_pn_valid = np.where(pn != -1)[0]
idx_tumsiz_missing = np.where(tumsiz == -1)[0]
idx_tumsiz_valid = np.where(tumsiz != -1)[0]
    
# set missing values to mean
# copy = false modifies array in place
imp_pn = Imputer(missing_values = -1, copy=False)
imp_pn.fit_transform(pn)
imp_tumsiz = Imputer(missing_values = -1, copy=False)
imp_tumsiz.fit_transform(tumsiz)


for i in np.arange(NUM_CYCLES):
    print("Cycle " + str(i+1))
    
    # fit pn
    X_ind = np.hstack((X_cat[idx_pn_valid], tumsiz[idx_pn_valid]))
    X_rep = np.hstack((X_cat[idx_pn_missing], tumsiz[idx_pn_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, pn[idx_pn_valid])
    pn[idx_pn_missing] = lin_reg.predict(X_rep)
    pn = np.maximum(pn, 0)
    pn = np.minimum(pn, 100)
    
    # fit tumsiz
    X_ind = np.hstack((X_cat[idx_tumsiz_valid], pn[idx_tumsiz_valid]))
    X_rep = np.hstack((X_cat[idx_tumsiz_missing], pn[idx_tumsiz_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, tumsiz[idx_tumsiz_valid])
    tumsiz[idx_tumsiz_missing] = lin_reg.predict(X_rep)
    tumsiz = np.maximum(tumsiz, 0)
    tumsiz = np.minimum(tumsiz, 990)


Cycle 1
Cycle 2
Cycle 3
Cycle 4
Cycle 5
Cycle 6
Cycle 7
Cycle 8
Cycle 9
Cycle 10


In [20]:
age_dx = np.array(X_cont[:,0])
malig = np.array(X_cont[:,4])
X = np.column_stack((X_cat_bin, age_dx, pn, malig, tumsiz))
print(X.shape)

(313122, 372)


We save the preprocessed data to disk. They are now available for our models.

In [21]:
if race == "white":
    np.save('../Data/white/X.npy', X)
    np.save('../Data/white/Y2.npy', Y2)
if race == "hispanic":
    np.save('../Data/hispanic/X.npy', X)
    np.save('../Data/hispanic/Y2.npy', Y2)
if race == "mixed":
    np.save('../Data/mixed/X.npy', X)
    np.save('../Data/mixed/Y2.npy', Y2)